In [14]:
%pylab inline
import pandas as pd
import numpy as np
from IPython.display import display
import re

Populating the interactive namespace from numpy and matplotlib


# DAS System

1. Total Picking 
total picking 시에는 차수에 있는 제품들을 모두 가져오라고 지시한다. 

2. Distribution
분배시에는 가져온 상품들을.. Random하게 바코드를 찍어서 어디에 넣을지를 알아냄. 

# Configurations

In [9]:
file_name = '/dataset/personal_projects/lsm/20160123_1차 발주_3280.xlsx'
SHEET_NAME = 'original'
CELL_SIZE = 96
DAS_

In [2]:
def display_time(seconds):
    response = ''
    hours = floor(seconds/int(60*60))
    seconds -= hours*60*60
    if hours >= 1:
        response += '%d시간 ' % hours
    
    minutes = floor(seconds/int(60))
    seconds -= minutes*60
    if minutes >= 1:
        response += '%d분 ' % minutes
    
    response += '%d초' % seconds
    return response


def display_result(n_epoch, n_location, n_product, n_barcode, is_possible=None):
    cost_location = n_location * FINDING_LOCATION_TIME
    cost_product = n_barcode * FINDING_PRODUCT_TIME
    if is_possible is not None:
        print('시뮬레이션 결과:\t', is_possible)
    
    print('차수 횟수:\t %d개' % n_epoch)
    print('길찾기 횟수:\t %d건' % n_location)
    print('상품찾기 횟수:\t %d건' % n_product)
    print('바코드찾기 횟수:\t %d건' % n_barcode)
    print('총 찾기 횟수:\t %d건' % int(n_location + n_barcode))
    print('갈찾기 소요시간:\t', display_time(cost_location))
    print('바코드찾기소요시간:\t', display_time(cost_product) )
    print('총 소요시간:\t', display_time(cost_location + cost_product))    

SyntaxError: invalid syntax (<ipython-input-2-45a7feee5c37>, line 6)

In [ ]:

class LSM(object):
    EMPTY_LOCATION = 'Z1-1'
    
    def __init__(self, DAS=96):
        self.data = None
        self.das_size = DAS
        
    def load(self, file_name, sheet=SHEET_NAME):
        COL_NAMES = ['date', 'order_number', 'brand_code', 'brand_name', 
                     'product', 'property', 'location', 'barcode', 'n', 'price', 
                     'status', 'orderer', 'recipient', 'address', 'postcode', 
                     'contact1', 'contact2', 'comment', 'part_delivery', 'code']
        self.data = pd.read_excel(file_name, 
                                  names=COL_NAMES,
                                  sheetname=sheet)
    
    def order_by_product(self):
        most_products = self.data[['product', 'n']].groupby(by='product').sum()
        most_products = most_products.sort_values(by='n', ascending=False)
        codes = []
        for product in most_products.index:
            for code in self.data[self.data['product'] == product]['code'].values:
                if code not in codes:
                    codes.append(code)
        
        codes = np.array(codes)
        return codes
    
    def preprocess(self, codes):
        
    

lsm = LSM()
lsm.load(file_name)
codes = lsm.order_by_product()
codes

In [20]:
class Cell(object):
    pass

class LSM(object):
    EMPTY_LOCATION = 'Z1-1'
    
    def __init__(self, cell_size):
        self.cells = [Cell() for _ in range(cell_size)]
    
    def load(self, file_name, sheet=SHEET_NAME):
        COL_NAMES = ['date', 'order_number', 'brand_code', 'brand_name', 
                     'product', 'property', 'location', 'barcode', 'n', 'price', 
                     'status', 'orderer', 'recipient', 'address', 'postcode', 
                     'contact1', 'contact2', 'comment', 'part_delivery', 'code']
        self.data = pd.read_excel(file_name, 
                                  names=COL_NAMES,
                                  sheetname=sheet)
        
    def preprocess(self, gte=2, lte=None):
        # Pre-Process missing locations
        self.data['location'] = self.data['location'].fillna(self.EMPTY_LOCATION)
        
        # Split Locations to l1, l2, l3
        location_regex = '(?P<l1>[a-zA-Z]+)-?(?P<l2>\d+)-(?P<l3>\d+)'
        self.data['location'][~self.data['location'].str.contains(location_regex)] = self.EMPTY_LOCATION
        location_series = self.data['location'].str.extract(location_regex)
        self.data = pd.concat([self.data, location_series], axis=1)
        
        # 
        print(self.data['barcode'])
    
lsm = LSM(cell_size=CELL_SIZE)
lsm.load(file_name)
lsm.preprocess()
# lsm.search(theta=0.2)

0       S144657
1       S144796
2       S127375
3       S136941
4       S118310
5       S131654
6       S128637
7       S144664
8       S137585
9       S118310
10      S118310
11      S141852
12      S120743
13      S127375
14      S140734
15      S127375
16      S120067
17      S127375
18      S134436
19      S141861
20      S145645
21      S127375
22      S145455
23      S127375
24      S142259
25      S142266
26      S141861
27      S136263
28      S145277
29      S138734
         ...   
3249     S76044
3250    S130333
3251     S76040
3252     S76042
3253    S102393
3254    S135578
3255    S143553
3256     S76046
3257    S146186
3258    S100817
3259    S105974
3260    S135148
3261    S135148
3262    S135152
3263    S135152
3264    S137489
3265    S145277
3266    S143553
3267    S146201
3268    S145454
3269    S138428
3270    S142779
3271    S142776
3272    S135956
3273    S100818
3274    S102898
3275    S139402
3276    S140266
3277    S141321
3278    S145639
Name: barcode, dtype: ob

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:26: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
